In [1]:
#imports
import dask, joblib
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask_ml import cluster, decomposition, linear_model, metrics
from dask_ml.impute import SimpleImputer
from dask_ml.model_selection import train_test_split, KFold
from dask_ml.preprocessing import OneHotEncoder, StandardScaler
from sklearn import tree, ensemble, svm
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import xgboost as xgb
import pandas as pd
import numpy as np

def cat_features(dataframe):
    td = pd.DataFrame({'a': [1,2,3], 'b': [1.0, 2.0, 3.0]})
    return filter(lambda x: not(dataframe[x].dtype in [td['a'].dtype, td['b'].dtype]), list(dataframe))

#close existing dask connection if it exists and open a new one
try:
    if client is not None:
        client.close()
        print("closed existing connection, ",client)       
except Exception as e:
    print(e)

name 'client' is not defined


In [2]:
#get new connection
client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
display(client)

/home/cgrosh/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:38427 Dashboard: http://127.0.0.1:34101/status,Cluster Workers: 2 Cores: 4 Memory: 16.00 GB


In [3]:
# Declare the dtypes of the file to be more efficient when reading in 
feat_types = {'Dst Port': np.float64, 'Protocol': np.int64, 'Timestamp': str, 
              'Flow Duration': np.int64, 'Tot Fwd Pkts': np.int64, 'Tot Bwd Pkts': np.int64, 
              'TotLen Fwd Pkts': np.int64, 'TotLen Bwd Pkts': np.int64, 'Fwd Pkt Len Max': np.int64, 
              'Fwd Pkt Len Min': np.int64, 'Fwd Pkt Len Mean': np.float64, 'Fwd Pkt Len Std': np.float64, 
              'Bwd Pkt Len Max': np.int64, 'Bwd Pkt Len Min': np.int64, 'Bwd Pkt Len Mean': np.float64, 
              'Bwd Pkt Len Std': str, 'Flow Byts/s': str, 'Flow Pkts/s': np.float64, 
              'Flow IAT Mean': np.float64, 'Flow IAT Std': np.float64, 'Flow IAT Max': np.int64, 
              'Flow IAT Min': np.int64, 'Fwd IAT Tot': np.int64, 'Fwd IAT Mean': np.float64, 
              'Fwd IAT Std': np.float64, 'Fwd IAT Max': np.int64, 'Fwd IAT Min': np.int64, 
              'Bwd IAT Tot': np.int64, 'Bwd IAT Mean': np.float64, 'Bwd IAT Std': np.float64, 
              'Bwd IAT Max': np.int64, 'Bwd IAT Min': np.int64, 'Fwd PSH Flags': np.int64, 
              'Bwd PSH Flags': np.int64, 'Fwd URG Flags': np.int64, 'Bwd URG Flags': np.int64, 
              'Fwd Header Len': np.int64, 'Bwd Header Len': np.int64, 'Fwd Pkts/s': np.float64, 
              'Bwd Pkts/s': np.float64, 'Pkt Len Min': np.int64, 'Pkt Len Max': np.int64, 
              'Pkt Len Mean': np.float64, 'Pkt Len Std': np.float64, 'Pkt Len Var': np.float64, 
              'FIN Flag Cnt': np.int64, 'SYN Flag Cnt': np.int64, 'RST Flag Cnt': np.int64, 
              'PSH Flag Cnt': np.int64, 'ACK Flag Cnt': np.int64, 'URG Flag Cnt': np.int64, 
              'CWE Flag Count': np.int64, 'ECE Flag Cnt': np.int64, 'Down/Up Ratio': np.int64, 
              'Pkt Size Avg': np.float64, 'Fwd Seg Size Avg': np.float64, 'Bwd Seg Size Avg': np.float64, 
              'Fwd Byts/b Avg': np.int64, 'Fwd Pkts/b Avg': np.int64, 'Fwd Blk Rate Avg': np.int64, 
              'Bwd Byts/b Avg': np.int64, 'Bwd Pkts/b Avg': np.int64, 'Bwd Blk Rate Avg': np.int64, 
              'Subflow Fwd Pkts': np.int64, 'Subflow Fwd Byts': np.int64, 'Subflow Bwd Pkts': np.int64, 
              'Subflow Bwd Byts': np.int64, 'Init Fwd Win Byts': np.int64, 'Init Bwd Win Byts': np.int64, 
              'Fwd Act Data Pkts': np.int64, 'Fwd Seg Size Min': np.int64, 'Active Mean': np.float64, 
              'Active Std': np.float64, 'Active Max': np.int64, 'Active Min': np.int64, 
              'Idle Mean': np.float64, 'Idle Std': np.float64, 'Idle Max': np.int64,
              'Idle Min': np.int64, 'Label': str}

feat_types_check = {'Dst Port': float, 'Protocol': int, 'Timestamp': str, 
              'Flow Duration': int, 'Tot Fwd Pkts': int, 'Tot Bwd Pkts': int, 
              'TotLen Fwd Pkts': int, 'TotLen Bwd Pkts': int, 
              'Flow Pkts/s': float,  'Fwd PSH Flags': int, 
              'Bwd PSH Flags': int, 'Fwd URG Flags': int, 'Bwd URG Flags': int, 
              'Fwd Pkts/s': float, 
              'Bwd Pkts/s': float,
              'FIN Flag Cnt': int, 'SYN Flag Cnt': int, 'RST Flag Cnt': int, 
              'PSH Flag Cnt': int, 'ACK Flag Cnt': int, 'URG Flag Cnt': int, 
              'CWE Flag Count': int, 'ECE Flag Cnt': int, 
              'Subflow Fwd Pkts': int, 'Subflow Fwd Byts': int, 'Subflow Bwd Pkts': int, 
              'Subflow Bwd Byts': int, 'Label': str}

feature_cols = ['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 
                'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
                'TotLen Bwd Pkts', 
                'Flow IAT Mean', 'Fwd PSH Flags', 'Bwd PSH Flags', 
                'Fwd URG Flags','Bwd URG Flags', 'FIN Flag Cnt', 
                'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 
                'ACK Flag Cnt', 'URG Flag Cnt','CWE Flag Count', 
                'ECE Flag Cnt', 'Subflow Fwd Pkts','Subflow Fwd Byts', 
                'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Label']
# print(len(feat_types_check))
# print(feature_cols[18])
df = dd.read_csv('cic_data/23_02_2018.csv', usecols=feature_cols, low_memory=False)
df_2 = dd.read_csv('cic_data/16_02_2018_fixeddata.csv', usecols=feature_cols, low_memory=False)
# df_3 = dd.read_csv('cic_data/02_03_2018.csv', usecols=feature_cols, low_memory=False)
df = df.append(df_2)
# df = df.append(df_3)
print("Length of Dataset: ", len(df))
print("Number of Columns: ", len(df.columns))
# print("Categorical Features: ", list(cat_features(df)))
# Files in the zipped folder:
# 23_02_2018.csv, 20_02_2018.csv, 16_02_2018.csv, 02_03_2018.csv
# with ZipFile('cic_data.zip') as zipped:
#     df = dd.from_pandas(pd.read_csv(zipped.open('23_02_2018.csv'), usecols=feature_cols), npartitions=2)
#     df = df.append(pd.read_csv(zipped.open('16_02_2018.csv'), usecols=feature_cols))

Length of Dataset:  2097149
Number of Columns:  26


In [4]:
x_cols = ['Dst Port', 'Protocol', 'Flow Duration', 
                'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
                'TotLen Bwd Pkts', 'Flow Byts/s', 'Flow Pkts/s', 
                'Flow IAT Mean', 'Fwd PSH Flags', 'Bwd PSH Flags', 
                'Fwd URG Flags','Bwd URG Flags', 'FIN Flag Cnt', 
                'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 
                'ACK Flag Cnt', 'URG Flag Cnt','CWE Flag Count', 
                'ECE Flag Cnt', 'Subflow Fwd Pkts','Subflow Fwd Byts', 
                'Subflow Bwd Pkts', 'Subflow Bwd Byts']
df['Target'] = 0 
df['Target'] = df['Target'].mask(df['Label'] != 'Benign', 1)

In [5]:
display(df.head())

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Flow IAT Mean,Fwd PSH Flags,...,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Label,Target
0,22,6,23/02/2018 08:18:29,1532698,11,11,1179,1969,7.298562e+04,0,...,0,0,0,0,11,1179,11,1969,Benign,0
1,500,17,23/02/2018 08:17:45,117573855,3,0,1500,0,5.878693e+07,0,...,0,0,0,0,3,1500,0,0,Benign,0
2,500,17,23/02/2018 08:17:45,117573848,3,0,1500,0,5.878692e+07,0,...,0,0,0,0,3,1500,0,0,Benign,0
3,22,6,23/02/2018 08:19:55,1745392,11,11,1179,1969,8.311390e+04,0,...,0,0,0,0,11,1179,11,1969,Benign,0
4,500,17,23/02/2018 08:18:17,89483474,6,0,3000,0,1.789669e+07,0,...,0,0,0,0,6,3000,0,0,Benign,0


In [6]:
y = df['Target']
# drop_cols = [col for col in df.columns if col not in x_cols]
x = df.drop(['Label', 'Target', 'Timestamp'], axis=1)

In [7]:
display(y.head())

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64

In [8]:
#function that takes X, Y and gives you a train test split
def doTrainTestSplit(X,Y):
    #breakpoint()
    #look at dask dataframes
#     display(X.head())
#     display(Y.head())
   
    #get splits
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.3)
#     display(X_train.compute())
#     display(y_train.compute())
    return X_train, X_test, y_train, y_test 

In [9]:
def doKFolds(X,Y):
    
    #look at dask dataframes
    display(X.head())
    display(yD.head())

    #create KFold object
    c = KFold()
    #breakpoint()
    #split on dask arrays, doesn't work on dataframes yet
    gen = c.split(X.to_dask_array(lengths=True),Y.to_dask_array(lengths=True))

    #inspect generator
    print(gen)
    display(type(gen))
    
    #call generator
    for train,test in gen:
        print("train = ",train.compute())
        print("test = ",test.compute())
        print("x train = ",X.loc[train])
        got = X.loc[train.compute()]
        display(got.head())
        #print(got.compute())
        clf.fit(got,got)


In [10]:
x_train, x_test, y_train, y_test = doTrainTestSplit(x,y)

In [11]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
scaler = StandardScaler()

In [13]:
from sklearn.metrics import precision_score, recall_score
xgb_clf = xgb.XGBClassifier(n_estimators=5, max_depth=10)
rf_clf= tree.DecisionTreeClassifier()

In [13]:
%%timeit -n 10
#time with just pandas
with joblib.parallel_backend('threading'):
     xgb_clf.fit(x_train.compute(), y_train.compute(), eval_metric=precision_score)

32.7 s ± 1.71 s per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
precision_score(y_test.compute(), xgb_clf.predict(x_test.compute()))

0.9835228642170074

In [15]:
%%timeit -n 10
#time with just pandas
with joblib.parallel_backend('dask'):
     xgb_clf.fit(x_train.compute(), y_train.compute(), eval_metric=precision_score)

32.3 s ± 2.48 s per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
xgb_clf.fit(x_train.compute(), y_train.compute(), eval_metric=precision_score)
rf_clf.fit(x_train.compute(), y_train.compute())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
precision_score(y_test.compute(), xgb_clf.predict(x_test.compute()))

0.9834634138314968

In [17]:
recall_score(y_test.compute(), rf_clf.predict(x_test.compute()))

0.9926953536024811

In [14]:
y_hat = xgb_clf.predict(x_test.compute())
confusion_matrix(y_test, y_hat, labels=[1,0])

array([[179701,   1205],
       [  2940, 446466]])

In [15]:
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))

0.9933390821752733